In [1]:
import mpmath as mp
import sympy as sp
sp.init_printing()

Define our parameters
---------------------

In [2]:
precision = 100

A = sp.Integer(12)

# A-1 / A+1 will have type sp.Rational. 
# Here we take a Float to given precision
α = ((A-1)/(A+1)).evalf(precision)**2
X = sp.log(1/α)
ξ = 1 + (α/(1-α))*sp.log(α)
E0 = 2E6 # 2 MeV
E = 1 # 1 eV
u = sp.log(E0/E)

In [3]:
%whos

Variable    Type       Data/Info
--------------------------------
A           Integer    12
E           int        1
E0          float      2000000.0
X           Float      0.33410816932633238398308<...>3469326490399108757093947
mp          module     <module 'mpmath' from '/h<...>ages/mpmath/__init__.py'>
precision   int        100
sp          module     <module 'sympy' from '/ho<...>kages/sympy/__init__.py'>
u           Float      14.5086577385242
α           Float      0.71597633136094674556213<...>5917159763313609467455621
ξ           Float      0.15776898982320378204263<...>6671072805452246674825675


Define symbolic solution
------------------------

In [4]:
def n_average_collisions(α, X, u):    
    i = sp.ceiling(u/X)
    k, m = sp.symbols("k, m")
    R_k = sp.Sum(
        # Summand
        (-1)**m * sp.factorial(k)/sp.factorial(m) * α**m*(1-α)**(k-m+1) * (u-k*X)**m,
        # Limits
        (m, 0, k-1)
    )

    R = sp.Piecewise(
        # R(k>0):
        ((R_k + (-1)**k*α**k*(u-k*X)**k) / (sp.factorial(k)*α**k*(1-α)**k), k > 0), 
        # R(k<=0):
        (1, k <= 0)
    )
    
    N = sp.Sum(
        # Summand
        1 + 1/α * (R*sp.exp((α*u-k*X)/(1-α)) - 1),
        # Limits
        (k, 0, i)
    )
    
    return N

Print analytical result
-----------------------

In [5]:
symbols = sp.symbols("α,X,u")
expr = n_average_collisions(*symbols)
expr

     ⎡u⎤                                                                      
     ⎢─⎥                                                                      
     ⎢X⎥                                                                      
_____________                                                                 
╲                                                                             
 ╲            ⎧                       ⎛                       k - 1           
  ╲           ⎪                       ⎜                        ____           
   ╲          ⎪                       ⎜                        ╲              
    ╲         ⎪                       ⎜                         ╲       m  m  
     ╲        ⎪          -k        -k ⎜    k  k           k      ╲  (-1) ⋅α ⋅(
      ╲       ⎪         α  ⋅(1 - α)  ⋅⎜(-1) ⋅α ⋅(-X⋅k + u)  +    ╱  ──────────
       ╲      ⎪                       ⎜                         ╱             
        ╲     ⎪                       ⎜             

## Approximation

In [6]:
0.7 + ξ**-1 * u

Evaluate result using mpmath
----------------------------
Passing in `n` implicitly uses mpmath, but we need to specify mpmath types for the constants

In [7]:
expr.evalf(n=precision, subs={symbols[0]: α,
                              symbols[1]: X,
                              symbols[2]: u})

---

Alternatively, using a direct approach
--------------------------------------

In [8]:
import mpmath as mp
mp.mp.dps = precision

def n_average_collisions_numerical(α, X, u):    
    i = mp.ceil(u/X)
    def term(k):
        return 1 + 1/α * (R(α, X, u, k)*mp.exp((α*u-k*X)/(1-α)) - 1)
    return mp.nsum(term, [0, i])

def R(α, X, u, k):
    if not k:
        return 1.0
    
    def term(m):
        return (-1)**m * mp.factorial(k)/mp.factorial(m) * α**m*(1-α)**(k-m+1) * (u-k*X)**m 
              
    return (mp.nsum(term, [0, k-1]) + (-1)**k*α**k*(u-k*X)**k) / (mp.factorial(k)*α**k*(1-α)**k)

Evaluate result, using mpmath types 
-----------------------------------
These types implement operator overloading, so behave numerically

In [9]:
n_average_collisions_numerical(α, X, u)

mpf('92.64722661238204283037055304094536823448065053367578269916537047019645037824813004574237313181058706566')